In [1]:
import pickle
import numpy as np
import pandas as pd

/cerebralcortex/kessel_jupyter_virtualenv/tensorflow/lib/python3.7/site-packages/pandas-1.2.3-py3.7-linux-x86_64.egg/pandas/compat/__init__.py:97: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
results = pickle.load(open('./data/walking_results_10.p','rb'))

In [52]:
data_col = []
for window_size,test_score,test_df,y_test,unique_users_dict in results:
#     if len(np.unique(y_test))<20:
#         continue
#     temp_df = test_df
    acc_ = accuracy_score(test_df['original'],test_df['prediction'])
    
    test_score = test_score.append(pd.DataFrame({'user':[-1],
                                    'majority_score':[acc_],
                                    'maxmean_score':[acc_],
                                    'test_lengths':[0]}))
    
    
    test_score['train_lengths'] = window_size
    
    
    
    
    
    data_col.append(test_score)
    

In [53]:
from sklearn.metrics import accuracy_score

In [54]:
scores = pd.concat(data_col)
scores = scores[scores.test_lengths.isin(list(np.arange(1,177,8))+[0])]

In [55]:
scores_mean = scores.groupby(['train_lengths','test_lengths'],as_index=False).mean()

In [56]:
maxmean_score = pd.pivot_table(scores_mean,values='maxmean_score',index='test_lengths',columns='train_lengths',aggfunc='mean')

try:
    import seaborn as sns
    import matplotlib.pyplot as plt
    plt.rcParams.update({'font.size':15})
    plt.figure(figsize=(20,10))
    sns.heatmap(maxmean_score,annot=True,fmt='.2f',cmap='Reds',linewidth=1,linecolor='black')
    plt.title('Max Mean Boosting')
    plt.xlabel('Train data per user, minutes')
    plt.ylabel('Test data length, minutes')
    plt.savefig('./images/maxmean_boosting.png',dps=1e6)
    plt.show()
except Exception as e:
    maxmean_score.to_json('maxmean_10.json')


In [57]:
maxmean_score = pd.pivot_table(scores_mean,values='majority_score',index='test_lengths',columns='train_lengths',aggfunc='mean')


try:
    import seaborn as sns
    import matplotlib.pyplot as plt
    plt.rcParams.update({'font.size':15})
    plt.figure(figsize=(20,10))
    sns.heatmap(maxmean_score,annot=True,fmt='.2f',cmap='Reds',linewidth=1,linecolor='black')
    plt.title('Majority Boosting')
    plt.xlabel('Train data per user, minutes')
    plt.ylabel('Test data length, minutes')
    plt.savefig('./images/majority_boosting.png',dps=1e6)
    plt.show()
except Exception as e:
    maxmean_score.to_json('majority_10.json')